In [7]:
import numpy as np
from music21 import *
import glob
import sys
import math
from tensorflow.keras.models import Sequential, Model, load_model
sys.path.append("../RL modules/")
from env_noOctave import SegmentationEnv

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html


In [12]:
base_lr = 0.001
max_lr = 0.1
step_size = 8
def masterScheduler(epoch):
    def scheduler():
            period = 2 * step_size
            cycle = math.floor(1 + epoch/period)
            x = abs(epoch/step_size - 2*cycle + 1)
            delta = (max_lr - base_lr)*max(0, (1 - x))
            delta /= float(2 ** (cycle - 1))
            return base_lr + delta
    return scheduler

In [13]:
for i in range(20):
    print(masterScheduler(i)())

0.001
0.013375000000000001
0.025750000000000002
0.038125000000000006
0.0505
0.062875
0.07525000000000001
0.08762500000000001
0.1
0.08762500000000001
0.07525000000000001
0.062875
0.0505
0.038125000000000006
0.025750000000000002
0.013375000000000001
0.001
0.0071875
0.013375000000000001
0.019562500000000003


In [22]:
testing_pieces = []
# for piece in glob.glob('./normal/training/*'):
#     testing_pieces.append(piece)

for piece in glob.glob('./C_normalized/testing/*'):
    testing_pieces.append(piece)

In [23]:
model = load_model("DQN-normal")
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
dense (Dense)                (None, 48)                1248      
_________________________________________________________________
dense_1 (Dense)              (None, 24)                1176      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 2,474
Trainable params: 2,474
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
env = SegmentationEnv(testing_pieces)

./C_normalized/testing\trans_Il_Vecchio_Castello.musicxml
./C_normalized/testing\trans_Minuet_in_G_Major.musicxml
./C_normalized/testing\trans_Nocturne_no._1.musicxml
./C_normalized/testing\trans_Prélude_in_B_Minor.musicxml
./C_normalized/testing\trans_Sonate_No._31.musicxml
./C_normalized/testing\trans_Étude_in_Gb_Major_Op25.musicxml
Total number of pieces 6


In [25]:
len(env.notes),len(testing_pieces)

(6, 6)

In [28]:
def Visualization(piece,offset): #offset assume sorted.
    c = converter.parse(piece)
    all_notes = []
    for el in c.recurse().notes:
        if el.lyric is not None:
            el.lyric = el.lyric.replace("♭", "b")
        all_notes.append([el.lyric, el, cal_offset(el)])
    b = sorted(all_notes, key=lambda x: x[0] if x[0] is not None else "ZZZ")
    b = sorted(b, key=lambda x: x[-1])
    for e in b:
        if len(offset) >0 and e[2] == offset[0]:
            del offset[0]
            e[1].addLyric("CUT")
    GEX = musicxml.m21ToXml.GeneralObjectExporter(c)
    out = GEX.parse()
    piecename = piece.split("\\")[-1]
    piecename = piece.split(".")[0]
    piecename = "Vis_"+piecename + ".musicxml"
    with open(output, "wb") as f:
        f.write(out)
    

In [26]:
for i in range(len(env.notes)):
    obs = env.reset(i)
    total_reward = 0
    max_reward = 0
    num_correct_segment = 0
    while True:
        obs = obs.reshape((1, 12*2+1))
        action = np.argmax(model.predict(obs))
        obs, reward, done, info = env.step(action)
        #env.render()
        total_reward += reward
        max_reward += 1
        if reward ==1 and action == 1:
            num_correct_segment += 1
        if done:
            break
    print(f"Piece {testing_pieces[i]}, total reward = {total_reward}/{max_reward}, correct segment = {num_correct_segment}")

Piece ./C_normalized/testing\trans_Il_Vecchio_Castello.musicxml, total reward = 61/205, correct segment = 77
Piece ./C_normalized/testing\trans_Minuet_in_G_Major.musicxml, total reward = 52/120, correct segment = 19
Piece ./C_normalized/testing\trans_Nocturne_no._1.musicxml, total reward = 173/451, correct segment = 52
Piece ./C_normalized/testing\trans_Prélude_in_B_Minor.musicxml, total reward = 55/79, correct segment = 26
Piece ./C_normalized/testing\trans_Sonate_No._31.musicxml, total reward = 53/305, correct segment = 68
Piece ./C_normalized/testing\trans_Étude_in_Gb_Major_Op25.musicxml, total reward = 51/105, correct segment = 66
